### urth-core-dataframe Examples

### Initialize declarative widgets and import dependencies

In [ ]:
import pandas as pd
import declarativewidgets as declwidgets

declwidgets.init()

In [ ]:
%%html
<link rel="stylesheet" type="text/css" href="bcard.css">
<link rel='import' href='urth_components/urth-viz-table/urth-viz-table.html'
        is='urth-core-import'>

### Example 1 - Accessing the data

Initial DataFrame:

In [ ]:
aDataFrame1 = pd.DataFrame([
        ["A", 1, "2013-01-01"],
        ["B", 2, "2013-01-02"]
    ], columns=["Letter", "Number", "Date"]
)
aDataFrame1['Date'] = pd.to_datetime(aDataFrame1['Date'])

#### Example 1.a Accessing the data using the `value` property

In [ ]:
%%html
<template is="dom-bind">
    <urth-core-dataframe ref="aDataFrame1" value="{{x}}"></urth-core-dataframe>
    <h4>{{x.columns}}</h4>
    <h4>{{x.columnTypes}}</h4>
    <template is="dom-repeat" items="{{x.data}}">
        <div>{{item.0}}, {{item.1}}, {{item.2}}</div>
    </template>
    
    <urth-viz-table datarows="{{ x.data }}" selection="{{sel}}" columns="{{ x.columns }}"></urth-viz-table>
</template>

#### Example 1.b Accessing the data using the `rows`, `columns`, and `columnTypes` properties

In [ ]:
%%html
<template is="dom-bind">
    <urth-core-dataframe ref="aDataFrame1" column-types="{{cTypes}}" rows="{{r}}" columns="{{c}}"></urth-core-dataframe>
    <h4>{{cTypes}}</h4>
    <h4>{{c}}</h4>
    <template is="dom-repeat" items="{{r}}">
        <div>{{item.0}}, {{item.1}}</div>
    </template>
</template>

#### Example 1.c Accessing the data using the `rows` property and setting `row-as-object`

In [ ]:
%%html
<template is="dom-bind">
    <urth-core-dataframe ref="aDataFrame1" rows="{{r}}" columns="{{c}}" row-as-object></urth-core-dataframe>
    <h4>{{c}}</h4>
    <template is="dom-repeat" items="{{r}}">
        <div>{{item.Letter}}, {{item.Number}}</div>
    </template>
</template>

#### Example 1.d Accessing the data using the `value` property within an object

In [ ]:
class objectWithDf():
    df = aDataFrame1
objWithDf = objectWithDf()

In [ ]:
%%html
<template is="dom-bind">
    <urth-core-dataframe ref="objWithDf.df" value="{{odf}}"></urth-core-dataframe>
    <h4>{{odf.columns}}</h4>
    <h4>{{odf.columnTypes}}</h4>
    <template is="dom-repeat" items="{{odf.data}}">
        <div>{{item.0}}, {{item.1}}, {{item.2}}</div>
    </template>
    
    <urth-viz-table datarows="{{ odf.data }}" selection="{{sel}}" columns="{{ odf.columns }}"></urth-viz-table>
</template>

### Example 2 - Getting data updates
`urth-core-dataframe` has several ways of staying up to date to changes to the DataFrame in the kernel

#### Example 2a - Manual refresh using a button
Note that the `auto` property is not active.

In [ ]:
%%html
<template is="dom-bind">
    <urth-core-dataframe id="df2a" ref="aDataFrame1" value="{{x}}"></urth-core-dataframe>
    <h4>{{x.columns}}</h4>
    <template is="dom-repeat" items="{{x.data}}">
        <div>{{item.0}}, {{item.1}}</div>
    </template>
    <button onClick="$('#df2a').get(0).refresh()">refresh</button>
</template>

Update the DataFrame by running the next cell, then press `refresh`:

In [ ]:
aDataFrame1 = pd.DataFrame([
        ["C", 1],
        ["D", 2]
    ], columns=["Letter", "Number"]
)

#### Example 2b - Automatic refresh using `auto` property

Initial DataFrame:

In [ ]:
aDataFrame2 = pd.DataFrame([
        ["A", 1],
        ["B", 2]
    ], columns=["Letter", "Number"]
)

In [ ]:
%%html
<template is="dom-bind">
    <urth-core-dataframe ref="aDataFrame2" value="{{x}}" auto></urth-core-dataframe>
    <h4>{{x.columns}}</h4>
    <template is="dom-repeat" items="{{x.data}}">
        <div>{{item.0}}, {{item.1}}</div>
    </template>
</template>

Update the DataFrame by running the next cell. The changes will update automatically:

In [ ]:
aDataFrame2 = pd.DataFrame([
        ["C", 1],
        ["D", 2]
    ], columns=["Letter", "Number"]
)

### Example 3 - Connecting to Spark DataFrames
urth-core-dataframe also supports Spark DataFrames

In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')
sqlContext = pyspark.SQLContext(sc)

In [ ]:
sparkDataFrame = sqlContext.createDataFrame(pd.DataFrame([
        ["Jane", "Doe","Scala Developer", "jane@ibm.com", "123-432-5431", "http://www.ibm.com/jane", "2013-01-01"], 
        ["John", "Doe","Spark Engineer", "john@ibm.com", "143-421-5411", "http://www.ibm.com/john", "2013-01-02"],
        ["Joe", "Smith","Product Manager", "joe@ibm.com", "123-732-8421", "http://www.ibm.com/joe", "2013-01-03"]
    ], columns=["First Name", "Last Name", "Role", "Email", "Phone Number", "Website", "Date"]
))
sparkDataFrame = sparkDataFrame.withColumn("Date", sparkDataFrame["Date"].cast("Date"))

In [ ]:
%%html
<template is="dom-bind">
    <urth-core-dataframe ref="sparkDataFrame" value="{{x}}" auto></urth-core-dataframe>
    
    <template is="dom-repeat" items="{{x.data}}">
      <div class="bcard">
        <div class="info">
            <div class="line full-name"><span>{{item.0}}</span> <span>{{item.1}}</span></div>
            <span class="line title">{{item.2}}</span>
            <span class="line phone-number">{{item.3}}</span>
            <span class="line email">{{item.4}}</span>
            <span class="line website">{{item.5}}</span>
        </div>
        <div class="logo"></div>
      </div>
    </template>
</template>

### Example 4 - Limiting data
The `limit` attribute allows you to control how much data to bring into the browser

In [ ]:
t = pd.DataFrame(list(range(1000)))
largeDataFrame = sqlContext.createDataFrame(t)

In [ ]:
%%html
<template is="dom-bind">
    <urth-core-dataframe ref="largeDataFrame" limit="5" value="{{x}}"></urth-core-dataframe>
    <h4>{{x.columns}}</h4>
    <template is="dom-repeat" items="{{x.data}}">
        <div>{{item.0}}</div>
    </template>
</template>

### Example 5 - Displaying DataFrames
A more complex example showing a custom rendering. Note that `auto` is active, so if you make changes to `aDataFrame3` by executing a cell, the cards will update automatically.

In [ ]:
import pandas as pd
aDataFrame3 = pd.DataFrame([
        ["Jane", "Doe","Pandas Guru", "jane@ibm.com", "123-432-5431", "http://www.ibm.com/jane"], 
        ["John", "Doe","Software Engineer", "john@ibm.com", "143-421-5411", "http://www.ibm.com/john"],
        ["Joe", "Smith","Product Manager", "joe@ibm.com", "123-732-8421", "http://www.ibm.com/joe"]
    ], columns=["First_Name", "Last_Name", "Role", "Email", "Phone_Number", "Website"]
)

In [ ]:
%%html
<template is="dom-bind">

    <urth-core-dataframe ref="aDataFrame3" rows="{{r}}" row-as-object auto></urth-core-dataframe>

    <template id="test" is="dom-repeat" items="{{r}}">
      <div class="bcard">
        <div class="info">
            <div class="line full-name"><span>{{item.First_Name}}</span> <span>{{item.Last_Name}}</span></div>
            <span class="line title">{{item.Role}}</span>
            <span class="line phone-number">{{item.Phone_Number}}</span>
            <span class="line email">{{item.Email}}</span>
            <span class="line website">{{item.Website}}</span>
        </div>
        <div class="logo"></div>
      </div>
    </template>
    
</template>


### Example 6 - Exploring DataFrames
explore - Enables visual exploration of different types of DataFrames.  For more information see [declarativewidgets_explorer](https://github.com/jupyter-incubator/declarativewidgets_explorer).
   

In [ ]:
%%html
<urth-core-channel name="a" debug><urth-core-channel>

In [ ]:
from declarativewidgets import explore
explore(aDataFrame1, channel='a', properties={'selection-as-object': True}, bindings={'selection': 'sel'})

### Example 7 - Debugging DataFrames
Sometimes, you just need a quick way to debug a DataFrame to see what it contains. Use the `debug` property to have the element display a simple table. This is particularly useful when query support show in [urth-core-query.ipynb](urth-core-query.ipynb).

In [ ]:
%%html
<urth-core-dataframe ref="aDataFrame3" debug></urth-core-dataframe>